In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from astropy.io import fits as pyfits
from astropy.table import Table

# Access eBOSS quasars data

In this notebook we show a simple way to retrive a small amount of data from the SDSS repository and explore what is those.  We might use this data in future classes, for now we will only explore them, and maybe do some plots...

(Adapted from the one originally made for MACSS 2019)

## PART 1: Get the data

You will need to download:
    - The quasar catalog https://data.sdss.org/sas/dr14/eboss/qso/DR14Q/  (~1Gb)
    - The spPlates files as many as required for the sample we will define later... but be careful and dot download everything as it is ~300Gb. 
    
You can download the files directly from https://data.sdss.org/sas/dr14/eboss/qso/DR14Q/  and https://data.sdss.org/sas/dr14/eboss/spectro/redux/v5_10_0/  digging in each plate directory. Or follow the notebook... 

In [2]:
#Define the URL for the data to download and the filename as well as the local directory where to be saved.  
DATA_URL="https://data.sdss.org/sas/dr14/eboss/qso/DR14Q/"
file="DR14Q_v4_4.fits"
local_dir="./"
local_file = os.path.join(local_dir,file)

In [8]:
#Simple instructions to download the catalog, it checks it don't exist...
#It can take a while so don't desperate.

if not os.path.exists(file):
        print("downloading DR14 quasar dataset from %s to %s"
              % (DATA_URL+file, local_file ))
        tmp = urllib.request.urlretrieve(DATA_URL+local_file, local_file)
        print ("Downloaded file"+local_file)
else:
    print("%s already exist"%(local_file))


./DR14Q_v4_4.fits already exist


 We have downloaded the qso catalog, but since it contains so many objects it is not viable we work with it in our laptops, so lets make a selection.  First lets load the catalog and see its content

In [9]:
catalog=Table.read('DR14Q_v4_4.fits')
catalog.colnames

ValueError: cannot reshape array of size 190305 into shape (526356,)

In [7]:
catalog

NameError: name 'catalog' is not defined

As you see there are about 500,000 objetcs, this is what is used for a real life analysis. For this day we'll use a 
few only... 

In [7]:
#select objects with THING_ID>0. Also select the qso's within the zmin, zmax redshift interval, defined so that
#we capture many lyman alpha quasars 
w=(catalog['THING_ID']>0) & (catalog['Z']>2) & (catalog['Z']<4) & (catalog['RA']!=catalog['DEC'])& (catalog['RA']>0) & (catalog['DEC']>0)
reduced_cat=catalog[w]
reduced_cat

NameError: name 'catalog' is not defined

 We still have about 200,000 objects. Because we have limited and download a few files only, lets order all this qso's by the PLATE in which they were observed. You could also select them by position, or other randomly. 

In [14]:
#Group the catalog by PLATE and select only first N objects you want to use... 
#Note I've used only 50 here but the idea would be to download many more... 
cat_by_plate=reduced_cat.group_by('PLATE')
small_cat=cat_by_plate[0:100]
small_cat

NameError: name 'reduced_cat' is not defined

This will be our small catalog to use. Let's see how the qso's are distributed compared to the full catalog.

In [15]:
#Lets plot the distribution in RA/DEC of the full catalog, and the small catalog
plt.scatter(cat_by_plate['RA'],cat_by_plate['DEC'],marker='.')
plt.scatter(small_cat['RA'],small_cat['DEC'],marker='.')
plt.xlabel('RA')
plt.ylabel('DEC')

NameError: name 'cat_by_plate' is not defined

Excercise: Make a few more plots showing how the full and small catalog are distributed in redshift, magnitude, etc... 

In [16]:
thing_id=small_cat['THING_ID']
fiberid=small_cat['FIBERID']
plate=small_cat['PLATE']
zqso=small_cat['Z']

NameError: name 'small_cat' is not defined

Now each of the spectra is located in specific files called spPlate-PLATE-MJD. So lets make a list of all the spPlates that we need 

In [17]:
#GET THE UNIQUE PLATES WE NEED TO DOWNLOAD FOR THE SELECTION ABOVE AND CONSTRUCT THE FILE NAMES WE NEED. 
plate_list=[]
for p,m in zip(small_cat['PLATE'],small_cat['MJD']):
    plate_list.append(str(p)+'/spPlate-'+str(p)+'-'+str(m)+'.fits')
plate_list=np.unique(plate_list)
plate_list

NameError: name 'small_cat' is not defined

Now lets download the files the same way we did with the catalog

In [18]:
#Takes some minutes... we will have this data on a 
plates_url='https://data.sdss.org/sas/dr14/eboss/spectro/redux/v5_10_0/'
plates_local='./spPlates/'

if not os.path.exists(plates_local):
        os.makedirs(plates_local)

for plate_ in plate_list:
    url_file = os.path.join(plates_url,plate_)
    local_file=plates_local+plate_.split("/")[1]
    if not os.path.exists(local_file):
        print("downloading spplate from %s to %s"
              % (url_file, local_file))
        tmp = urllib.request.urlretrieve(url_file, local_file)
    else:
        print('%s  %s present on disk. '%(url_file,local_file))


If downloading multiple files, as many as to have a about 10K qso's, maybe is better to save the filenames list to a file 
and use it to download the files from terminal... it could be a little faster...

## PART 2: Explore the qso data. 

Lets explore what is in those file we've downloaded. For a full description of the spPlates look 
https://data.sdss.org/datamodel/files/BOSS_SPECTRO_REDUX/RUN2D/PLATE4/spPlate.html  

In [19]:
#Explore the fist of the spPlates.
file=plates_local+plate_list[2].split("/")[1]
plate1=pyfits.open(file)
print(file)

IndexError: list index out of range

In [ ]:
plate1.info()

In [20]:
#What is in the PLUGMAP
plugmap=plate1['PLUGMAP'].data
Table(plugmap)

NameError: name 'plate1' is not defined

As you can see an spPlate has more than only qso's... so we need to retrive only the information for the quasars we have selected in the small catalog. Then lets to this for the first spPlate. 

In [21]:
#Select from small catalog only qso's that are in first spPlate.
thisplate=plate_list[2].split("/")[0]
w=plate==int(thisplate)
ids_=fiberid[w]
zqso_=zqso[w]

IndexError: list index out of range

In [22]:
#Select from the plugmap nly the qso's that are in the small catalog. 
w=np.in1d(plugmap['FIBERID'],ids_)
small_plugmap=plugmap[w]
Table(small_plugmap)

NameError: name 'plugmap' is not defined

So far so good, but haven't selected the spectra, only the information about the qso's. So lets get the observed flux for the qso's, i.e. the spectra!

The wavelenght is given by the parameters 'COEFF0' and 'COEFF1' present in the header of the first hdu of the spPlate file, while the flux is an array in the same hdu.

In [23]:
#Get the wavelenght
pltheader=plate1[0].header
coeff0=pltheader['COEFF0']
coeff1=pltheader['COEFF1']


NameError: name 'plate1' is not defined

In [ ]:
#Get the spectra
flux=plate1[0].data
ivar=plate1[1].data
logwave=coeff0+coeff1*np.arange(flux.shape[1])

Lets look at what we got. 

In [24]:
#Plot some of the specta
plt.figure(figsize=(6,40),dpi=100)
i=0
for i in range(4):
    plt.subplot(len(ids_),1,i+1)
    plt.plot(10**logwave,flux[ids_[i]-1])
    plt.ylabel('Flux 1E-17 erg/cm^2/s/Ang')
    plt.xlabel('\lambda [\Ang]')
    plt.xlim(3500,6000)
    i+=1


NameError: name 'ids_' is not defined

<Figure size 600x4000 with 0 Axes>

Exercise: Ideally you'll do both but you can start with the one is more interesting for you. 

- For those interested in quasars :D

    - Do they look like Lya quasars?
    - Plot some of the spectra in the rest frame
    - Cut the spectra to save only the Lya region defined between 104-120nm in the rest frame
    - For each spectra save the zqso,wave,flux_lya,ivar, and save all in a single file. 
    - Make other selections criteria, to see other types of quasars... 
 
- For those just practicing/playing with the data 

    - Find other variables that could be linked between the spPlates and the DR14Q file. 
    - Play making different selections, maybe based in redshift, or color, or different combinations.
    - From the spPlates, what other types of objets are? make a descriptive analysis of these data, and the DR14Q file. 